# Tarea 3

- Paula Cintron 20160090
- Lunes 11 de agosto 2019



Ejercicios

1. Redefinan el model a w2 * t_u ** 2 + w1 * t_u + b
2. Que partes del training loop necesitaron cambiar para acomodar el nuevo modelo?
3. Que partes se mantuvieron iguales?
4. El loss resultante es mas alto o bajo despues de entrenamiento?
5. El resultado es mejor o peor?


Se necesitaron agregar dimensiones para que el nuevo modelo encajara a los dos pesos agregados. Se mantuvo igual la arquitectura original del modelo. Solo se le agregaron variables y se reestructuró el modelo a un modelo multivariable. El loss resultó ser mas bajo. El resultado es mejor dado que al entrenar el modelo se adapta a los parámetros dados (menor learning rate y no epochs). 

### 1. Redefinan el model a w2 * t_u ** 2 + w1 * t_u + b

In [140]:
import numpy as np
import torch

Para empezar, quiero repasar el código del modelo hecho en clase para luego construir sobre él y modificarlo.
Este es el modelo del ejercicio hecho en clase. 

In [141]:
#Definiendo las variables

t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0] # Temperatura en grados celsios
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4] # Unidades desconocidas
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [142]:
#Definiendo el modelo inicial

def model(t_u, w, b):
    return w * t_u + b


def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()


In [143]:
#Definiendo los parámetros
w = torch.ones(1)
b = torch.zeros(1)

t_p = model(t_u, w, b)
t_p


tensor([35.7000, 55.9000, 58.2000, 81.9000, 56.3000, 48.9000, 33.9000, 21.8000,
        48.4000, 60.4000, 68.4000])

In [144]:
loss = loss_fn(t_p, t_c)
loss

tensor(1763.8846)

In [145]:
#Redefiniendo el parámetro w 
# w = gradient descent 
# redefiniendo w1 

#Estos son el w y b dados del ejercicio hecho en clase 
delta = 0.1

loss_rate_of_change_w = (loss_fn(model(t_u, w + delta, b), t_c) -
                        loss_fn(model(t_u, w - delta, b), t_c)) / (2.0 * delta)

In [146]:
learning_rate = 1e-2

w = w - learning_rate * loss_rate_of_change_w

In [147]:
loss_rate_of_change_b = (loss_fn(model(t_u, w, b + delta), t_c) -
                        loss_fn(model(t_u, w, b - delta), t_c) / (2.0 * delta))

b = b - learning_rate * loss_rate_of_change_b

In [148]:
#Versión analítica 
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c) ** 2
    return squared_diffs.mean()

In [149]:
def dloss_fn(t_p, t_c):
    dsq_diffs = 2 * (t_p - t_c)
    return dsq_diffs

In [150]:
def model(t_u, w, b):
    return w * t_u + b


def dmodel_dw(t_u, w, b):
    return t_u


def dmodel_db(t_u, w, b):
    return 1.0

In [151]:

def grad_fn(t_u, t_c, t_p, w, b):
    dloss_dw = dloss_fn(t_p, t_c) * dmodel_dw(t_u, w, b)
    dloss_db = dloss_fn(t_p, t_c) * dmodel_db(t_u, w, b)
    return torch.stack([dloss_dw.mean(), dloss_db.mean()])

In [152]:
# Armando el training loop 
def training_loop(model, n_epochs, learning_rate, params, t_u, t_c, print_params=True):
    for epoch in range(1, n_epochs + 1):
        w, b = params
        
        t_p = model(t_u, w, b) # Forward pass
        loss = loss_fn(t_p, t_c)
        grad = grad_fn(t_u, t_c, t_p, w, b) # Backward pass
        
        params = params - learning_rate * grad
        
        if epoch % 10 == 0 or epoch == 1:
            print(f"Epoch {epoch}, Loss {loss}")
            if print_params:
                print(f"\tParams: {params}")
                print(f"\tGrad: {grad}")
        
    return params

In [153]:
#Corriendo el traning loop
training_loop(model,
              n_epochs = 50,
              learning_rate= 1e-2,
              params = torch.tensor([1.0, 0.0]),
              t_u = t_u,
              t_c = t_c)

Epoch 1, Loss 1763.8846435546875
	Params: tensor([-44.1730,  -0.8260])
	Grad: tensor([4517.2964,   82.6000])
Epoch 10, Loss 9.090107547845813e+34
	Params: tensor([3.2144e+17, 5.6621e+15])
	Grad: tensor([-3.2700e+19, -5.7600e+17])
Epoch 20, Loss inf
	Params: tensor([1.3457e+35, 2.3704e+33])
	Grad: tensor([-1.3690e+37, -2.4114e+35])
Epoch 30, Loss nan
	Params: tensor([nan, nan])
	Grad: tensor([nan, nan])
Epoch 40, Loss nan
	Params: tensor([nan, nan])
	Grad: tensor([nan, nan])
Epoch 50, Loss nan
	Params: tensor([nan, nan])
	Grad: tensor([nan, nan])


tensor([nan, nan])

# 1. Redefinan el model a w2 * t_u ** 2 + w1 * t_u + b

Voy a armar el modelo usando w1 y w2 como dos pesos individuales diferentes. Sin embargo, ambos seran matrices de 1 pero con diferentes pesos de delta.  

In [154]:
#Definiendo las variables

t_c1 = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0] # Temperatura en grados celsios
t_u1 = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4] # Unidades desconocidas
t_c1 = torch.tensor(t_c1)
t_u1 = torch.tensor(t_u1)
# Normalizando la data
t_u1 = 0.1 * t_u1

In [155]:
t_u1**2 + t_u1

tensor([16.3149, 36.8381, 39.6924, 75.2661, 37.3269, 28.8021, 14.8821,  6.9324,
        28.2656, 42.5216, 53.6256])

In [156]:

#Definiendo el modelo 2

def model2(t_u1, w1, w2, b):
    return w2 * t_u1 ** 2 + w1 * t_u1 + b


def loss_fn(t_p1, t_c1):
    squared_diffs = (t_p1 - t_c1)**2
    return squared_diffs.mean()

#def model3(t_u1,w3,b):
   # return w3 * t_u1 **2 + w3 * t_u1 +b

In [157]:
# Definiendo los parámetros
#Definiendo los parámetros
w1 = torch.ones(1)
w2 = torch.ones(1)
b = torch.zeros(1)
#w3 =torch.ones([2,1])

t_p1 = model2(t_u1, w1, w2, b)
t_p1



tensor([16.3149, 36.8381, 39.6924, 75.2661, 37.3269, 28.8021, 14.8821,  6.9324,
        28.2656, 42.5216, 53.6256])

In [158]:
#t_p2 = model3(t_u1,w3,b)
#t_p2

In [159]:
w2

tensor([1.])

In [160]:
w1

tensor([1.])

In [161]:
# Ver el resultado Loss de este nuevo modelo.
loss = loss_fn(t_p1, t_c1)
loss

tensor(675.7943)

In [162]:
# Armando el entrenamiento del nuevo modelo

#Estos son el w y b nuevos
delta2 = 0.1

loss_rate_of_change_w1 = (loss_fn(model2(t_u1, w1 + delta2,w2, b), t_c1) -
                        loss_fn(model2(t_u1, w1 - delta2,w2, b), t_c1)) / (2.0 * delta2)
delta3 = 0.2

loss_rate_of_change_w2 = (loss_fn(model2(t_u1, w1 ,w2 + delta3,b), t_c1) -
                        loss_fn(model2(t_u1, w1,w2 - delta3, b ), t_c1)) / (2.0 * delta3)

In [163]:
#Definiendo w1 y w2 

learning_rate = 1e-4

w1 = w1 - learning_rate * loss_rate_of_change_w1


w2 = w2 - learning_rate * loss_rate_of_change_w2

In [164]:
w1

tensor([0.9720])

In [165]:
w2

tensor([0.8260])

In [166]:
delta1= 0.1
loss_rate_of_change_b = (loss_fn(model2(t_u1, w1,w2,  b + delta1), t_c1) -
                        loss_fn(model2(t_u1, w1, w2, b - delta1), t_c1) / (2.0 * delta))

b = b - learning_rate * loss_rate_of_change_b

In [167]:
b

tensor([0.1584])

In [168]:
#Parte analítica
def loss_fn(t_p1, t_c1):
    squared_diffs = (t_p1 - t_c1) ** 2
    return squared_diffs.mean()

In [169]:
def dloss_fn(t_p1, t_c1):
    dsq_diffs = 2 * (t_p1 - t_c1)
    return dsq_diffs

In [170]:
def model2(t_u1, w1, w2, b):
    return w2*(t_u1**2) + (w1*t_u1) + b


def dmodel_dw(t_u1, w1, w2, b):
    return t_u


def dmodel_db(t_u1, w1, w2, b):
    return 1.0

In [171]:

def grad_fn(t_u1, t_c1, t_p1, w1, w2, b):
    dloss_dw = dloss_fn(t_p1, t_c1) * dmodel_dw(t_u1, w1, w2, b)
    dloss_db = dloss_fn(t_p1, t_c1) * dmodel_db(t_u1, w1, w2, b)
    return torch.stack([dloss_dw.mean(), dloss_dw.mean(),dloss_db.mean()])

## Primer cambio: esta funcion ahora tiene que retornar un tensor con 3 argumentos para que se pueda hacer la multiplicación mas adelante. Dado que se ingresa  t_u1 dos veces, retornaremos esa perdida dos veces

In [172]:
# Armando el training loop 
def training_loop(model2, n_epochs, learning_rate, params, t_u1, t_c1, print_params=True):
    for epoch in range(1, n_epochs + 1):
        w1, w2, b = params
        
        t_p1 = model2(t_u1, w1, w2, b) # Forward pass
        loss = loss_fn(t_p1, t_c1)
        grad = grad_fn(t_u1, t_c1, t_p1, w1, w2, b) # Backward pass
        
        params = params - learning_rate * grad
        print(params)
        
        if epoch % 10 == 0 or epoch == 1:
            print(f"Epoch {epoch}, Loss {loss}")
            if print_params:
                print(f"\tParams: {params}")
                print(f"\tGrad: {grad}")
        
    return params

In [173]:
learning_rate

0.0001

## Segundo cambio: se le agrega un nuevo parametro al tensor de params, para poder realizar la multiplicacion con el cambio que se hizo en grad_fn

In [174]:
#Corriendo el traning loop
#learning rate lo volví más pequeño que el original para que aprenda en gradientes mas pequeños
#Se normalizó la data previo a correr el modelo
training_loop(model2,
              n_epochs = 50,
              learning_rate= 1e-5,
              params = torch.tensor([1.0,0.0,1]),
              t_u1 = t_u1,
              t_c1 = t_c1)

tensor([1.0067, 0.0067, 1.0001])
Epoch 1, Loss 70.72434997558594
	Params: tensor([1.0067, 0.0067, 1.0001])
	Grad: tensor([-672.5403, -672.5403,   -8.6400])
tensor([1.0132, 0.0132, 1.0002])
tensor([1.0193, 0.0193, 1.0002])
tensor([1.0253, 0.0253, 1.0003])
tensor([1.0309, 0.0309, 1.0004])
tensor([1.0364, 0.0364, 1.0005])
tensor([1.0416, 0.0416, 1.0005])
tensor([1.0466, 0.0466, 1.0006])
tensor([1.0514, 0.0514, 1.0006])
tensor([1.0560, 0.0560, 1.0007])
Epoch 10, Loss 46.141624450683594
	Params: tensor([1.0560, 0.0560, 1.0007])
	Grad: tensor([-458.7160, -458.7160,   -5.0851])
tensor([1.0604, 0.0604, 1.0007])
tensor([1.0646, 0.0646, 1.0008])
tensor([1.0686, 0.0686, 1.0008])
tensor([1.0725, 0.0725, 1.0008])
tensor([1.0762, 0.0762, 1.0009])
tensor([1.0797, 0.0797, 1.0009])
tensor([1.0831, 0.0831, 1.0010])
tensor([1.0864, 0.0864, 1.0010])
tensor([1.0895, 0.0895, 1.0010])
tensor([1.0925, 0.0925, 1.0010])
Epoch 20, Loss 33.11180114746094
	Params: tensor([1.0925, 0.0925, 1.0010])
	Grad: tensor([-2

tensor([1.1423, 0.1423, 1.0011])

In [175]:
#params = training_loop(model2,
                       n_epochs = 5000,
                       learning_rate= 1e-4,
                       params = torch.tensor([1.0, 0.0]),
                       t_u1 = t_u1,
                       t_c1 = t_c1,
                       print_params=False)

#params

ValueError: not enough values to unpack (expected 3, got 2)